# Pathfinder for Hidden connections 

#https://neo4j.com/docs/cypher-manual/current/introduction/uniqueness/ 
#https://stackoverflow.com/questions/14345555/return-unique-nodes-in-cypher-path-query
#https://github.com/neo4j/neo4j/issues/12628
#How to improve it to just get once Episode I: The Phantom Menace?

In [1]:
#Imports
from neo4j import GraphDatabase, basic_auth
import neo4jupyter
import pandas as pd
from vis_class import vis_class

In [2]:
#Create db connector
# login = open("Login.txt", "r")
# uri = login.readline()
# user = login.readline()
# password = login.readline()
# login.close()
uri = "bolt://localhost:7687"
user = "neo4j"
password = "neo4j"

driver = GraphDatabase.driver(uri, auth=basic_auth(user, password))
session = driver.session(database="neo4j")

## General Path finding / matching


Here we get an short overview how characters are connected over max. 5 diffrent relations

In [3]:
five_path = '''MATCH p = ()-[*5]-() RETURN nodes(p) Limit 5'''
results = session.read_transaction(
    lambda tx: tx.run(five_path).data())
results

[{'nodes(p)': [{'name': 'SNAP'},
   {'name': 'Episode VII: The Force Awakens'},
   {'name': 'REY'},
   {'name': 'KYLO REN'},
   {'name': 'LIEUTENANT MITAKA'},
   {'name': 'Episode VII: The Force Awakens'}]},
 {'nodes(p)': [{'name': 'SNAP'},
   {'name': 'Episode VII: The Force Awakens'},
   {'name': 'REY'},
   {'name': 'KYLO REN'},
   {'name': 'LIEUTENANT MITAKA'},
   {'name': 'GENERAL HUX'}]},
 {'nodes(p)': [{'name': 'SNAP'},
   {'name': 'Episode VII: The Force Awakens'},
   {'name': 'REY'},
   {'name': 'KYLO REN'},
   {'name': 'CAPTAIN PHASMA'},
   {'name': 'FINN'}]},
 {'nodes(p)': [{'name': 'SNAP'},
   {'name': 'Episode VII: The Force Awakens'},
   {'name': 'REY'},
   {'name': 'KYLO REN'},
   {'name': 'CAPTAIN PHASMA'},
   {'name': 'HAN'}]},
 {'nodes(p)': [{'name': 'SNAP'},
   {'name': 'Episode VII: The Force Awakens'},
   {'name': 'REY'},
   {'name': 'KYLO REN'},
   {'name': 'CAPTAIN PHASMA'},
   {'name': 'GENERAL HUX'}]}]

## Finding hidden path

Something like that should it be
match (m:Movie) 
where m.name="Episode VI: Return of the Jedi" 
match (p:Person) 
where p.name="KI-ADI-MUNDI" 
match  
searchPath=(Movie)<-[*]-(Person)-[*]->(Movie)-[*]-(Person)-[*]->(Person) 
return distinct m.name,searchPath 


'''match (m:Movie) 
                    where m.name="Episode II: Attack of the Clones" 
                  match (p:Person) 
                    where p.name="KI-ADI-MUNDI" 
                  match searchPath=(Movie)<-[*]-(Person)-[*]->(Movie)-[*]-(Person)-[*]->(Person) 
                    return searchPath'''

In [ ]:
person_movie = '''match searchPath=(ms:Movie)<-[*]-()-[*]->(ps:Person) 
                    WHERE ms.name="Episode II: Attack of the Clones" AND ps.name="KI-ADI-MUNDI" 
                    return DISTINCT searchPath'''

results = session.read_transaction(
    lambda tx: tx.run(person_movie).data())
results

In [ ]:
person_movie = '''match searchPath=(ms:Movie)<-[*]->(ps:Person) 
                    WHERE ms.name="Episode II: Attack of the Clones" AND ps.name="KI-ADI-MUNDI" 
                    return DISTINCT searchPath'''

results = session.read_transaction(
    lambda tx: tx.run(person_movie).data())
results

In [ ]:
#Find path between Person and Movie
person_movie = '''match (m:Movie) 
                    where m.name="Episode II: Attack of the Clones" 
                  match (p:Person) 
                    where p.name="KI-ADI-MUNDI" 
                  match searchPath=(Movie)<-[*]-(Person)-[*]->(Movie)-[*]-(Person)-[*]->(Person) 
                    return searchPath'''
results = session.read_transaction(
    lambda tx: tx.run(person_movie).data())


In [4]:
# Paths between unconnected Person - Movie (Max 4 relations per path)
person_movie = '''match searchPath=(ms:Movie)<-[*..4]->(ps:Person) 
                    WHERE ms.name="Episode II: Attack of the Clones" AND ps.name="KI-ADI-MUNDI" 
                    return DISTINCT searchPath'''

results = session.read_transaction(
    lambda tx: tx.run(person_movie).data())
results

[{'searchPath': [{'name': 'Episode II: Attack of the Clones'},
   'APPEARS_IN',
   {'name': 'BAIL ORGANA'},
   'SPEAKS_WITH',
   {'name': 'MACE WINDU'},
   'SPEAKS_WITH',
   {'name': 'RABE'},
   'SPEAKS_WITH',
   {'name': 'KI-ADI-MUNDI'}]},
 {'searchPath': [{'name': 'Episode II: Attack of the Clones'},
   'APPEARS_IN',
   {'name': 'BAIL ORGANA'},
   'SPEAKS_WITH',
   {'name': 'MACE WINDU'},
   'SPEAKS_WITH',
   {'name': 'OBI-WAN'},
   'SPEAKS_WITH',
   {'name': 'KI-ADI-MUNDI'}]},
 {'searchPath': [{'name': 'Episode II: Attack of the Clones'},
   'APPEARS_IN',
   {'name': 'BAIL ORGANA'},
   'SPEAKS_WITH',
   {'name': 'MACE WINDU'},
   'APPEARS_IN',
   {'name': 'Episode I: The Phantom Menace'},
   'APPEARS_IN',
   {'name': 'KI-ADI-MUNDI'}]},
 {'searchPath': [{'name': 'Episode II: Attack of the Clones'},
   'APPEARS_IN',
   {'name': 'BAIL ORGANA'},
   'SPEAKS_WITH',
   {'name': 'MACE WINDU'},
   'SPEAKS_WITH',
   {'name': 'ANAKIN'},
   'SPEAKS_WITH',
   {'name': 'KI-ADI-MUNDI'}]},
 {'searc

In [5]:
# Shortest path between unconnected Person - Movie (Max 4 relations per path)
# subquery to find the shortest and then query to get the one with the shortest length
person_movie = '''
    CALL {
        match p=(ms:Movie)<-[*..4]->(ps:Person)
        WHERE ms.name="Episode IV: A New Hope" AND ps.name="KI-ADI-MUNDI"
        return min(length(p)) AS shortest
    }
    MATCH searchPath=(ms:Movie)<-[*..4]->(ps:Person)
    WHERE ms.name="Episode IV: A New Hope" AND ps.name="KI-ADI-MUNDI" AND length(searchPath) = shortest
    return searchPath
'''
results = session.read_transaction(
    lambda tx: tx.run(person_movie).data())
results

[{'searchPath': [{'name': 'Episode IV: A New Hope'},
   'APPEARS_IN',
   {'name': 'OBI-WAN'},
   'SPEAKS_WITH',
   {'name': 'KI-ADI-MUNDI'}]}]

In [ ]:
#Find path between Person and Person which are not related at the first glance

In [ ]:
#Find path between Movie and Movie which are not related at the first glance

## Closing the connection to the database

In [ ]:
#Close all connections
driver.close()
session.close()
drive = None
session = None